# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [8]:
from openai import OpenAI

client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图显示了一条木制的步道穿行于一片广阔的绿色草地中。步道直直地伸展，似乎引导着观赏者走向远方的自然风光。草地两侧生长着密集的高草和一些灌木，给人一种野生而未经开发的感觉。天空是晴朗的，有着几朵轻盈的白云，增添了一份宁静和广阔的氛围。\n\n整体上，这幅图表现了一种平和而美丽的自然景观，是徒步旅行和自然观察的理想场所。图中的色彩饱和且丰富，从鲜亮的绿色到淡蓝的天空，构成了一幅吸引人的自然风光画面。这样的场景可以让人感受到大自然的宁静与和谐。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [13]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [14]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图用幽默的方式展示了两种不同的狗，左边是一只肌肉发达的壮狗，右边是一只看起来相对普通、略显悠闲的狗。图中的文字在对比这两只狗的特质，左边的狗用“16岁的我——工作后的我”来形容，凸显其强壮和活力。文字还包括：“我刚读一片光盘，身体素质好，人世间的枭雄。”右边的狗则以“好想不眠 看电视动漫”以及“生活没有压力 错误的职业道路，饮食习惯差，精神状态差”来形容，表现了一种更加消极、缺乏动力的状态。

这种对比展示了在不同生活阶段、压力和环境影响下，同一个人（或在此图中指狗）可能呈现出的截然不


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [26]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")

def query_base64_image_description(image_path, prompt="解释下这台车是什么车,哪一款？是不是改装过?颜色是?", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [27]:
content = query_base64_image_description("./images/aaa.jpg")
print(content)

这台车是一辆标致（Peugeot）轿车，从车头的设计来看，可能是标致307或类似款式。这款车拥有较为典型的标致前端设计，包括品牌特有的前格栅和标志。

从图片中看，车辆似乎保持了原厂设计，没有明显的改装迹象。车的颜色是白色。

具体的车型和配置可能需要更多的细节或车辆其他角度的图片来进一步确认。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [22]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本里手写的笔记，内容主要关于人工智能语言模型的调优策略，针对的是P-Tuning、Prompt Tuning和LoRA技术。

1. **Prompt Tuning (Prompt Tuning ( FMT & Small Model ))**：这段笔记解释了Prompt Tuning用于小型模型的方法。Prompt Tuning通常指在固定预训练模型的参数的前提下，通过调整输入到模型的少量“prompts”（提示符），从而使模型更好地适应特定任务。在笔记中提到输入X由一系列的embedding后的tokens组成，并输出Y。

2. **Prefix Tuning**：这里详细介绍了Prefix Tuning策略，在训练过程中，通过添加前缀（一组可学习的向量）来调节模型行为，使模型输出期望的Y。其中提到了这些前缀是通过Transformer结构的Encoder/Decoder处理的。

3. **LoRA (Low-Rank Adaptation)**： 介绍了LoRA技术，它是一种模型参数调优的方法，通过低秩逼近来调整模型的权重矩阵W。通过增加一个小的权重更新ΔW，这个更新通过两个矩阵A和B的乘积得到，用以实现模型的细粒度调整，最后输出的模型表达为(W+ΔW)X，体现了通过低秩矩阵实现的调整效果。笔记还提到了Q-LORA和相关的存储需求变化（例如LoRA模型需要78GB，Q-LORA则需要48GB存储）。

整体来看，这张图片中的笔记内容非常专业，涉及到了深度学习和自然语言处理领域内的高级技术概念。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [31]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的不是车子，而是一本笔记本的一页，上面记载着与人工智能相关的术语和说明。图中的笔记包括了关于变换器（Transformers），PEFT、SOTA等技术的内容细节。不涉及任何关于车辆的信息，也没有提到颜色或改装等方面的内容。如果有其他问题或需要解答其他图片内容，请提供更多细节或上传相关图片。

![](./images/handwriting_1.jpg)

In [30]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

NameError: name 'Markdown' is not defined

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [ ]:
第一个作业:

In [28]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本的一页，记录的内容与车辆无关，而是关于机器学习和深度学习模型的一些笔记。文字描述涉及到Prompt Tuning, Prefix Tuning, LoRA等机器学习术语，并探讨了这些技术如何应用于自然语言处理中的模型。图片中没有显示任何车辆或其他与车辆相关的信息。


In [ ]:
第二个作业

In [32]:
content = query_base64_image_description("./images/handwriting_1.jpg",prompt="请解释一下这个笔记,并且用markdown格式输出一下")
display(Markdown(content))

这是一份关于深度学习模型（特别是Transformers模型）的笔记，重点在于模型调节（tuning）与适配方法（adapters）。笔记记录了多种方法和技术，以及它们的相关源头（例如，来自Google或Stanford的研究）。此外，也提到了某些特定类型的模型融合与改进方案。

以下是以markdown格式整理的部分内容概览：

### 转换器模型与基准
- **Transformers**：高效性能
- **Benchmark**：性能测试标准

### 模型微调方法
- **PEFT/SOTA**: 现有最佳方法
- **PBFT Methods**: 模型具体微调技术

### PEFT细节
- **Prompt Tuning**：
  - Adapter (2019, Google)
  - Prefix (2021, Stanford)
  - Prompt (2021, Google)
  - P-Tuning v1 (2021)
  - P-Tuning v2 (2022)
  - Soft Prompts (2021)
  - Hand Prompts (template-based)
- **Instruction Tuning for LLMs**:
  - Google API
  - 等等

### 新方法与改进
- **LoRA**: 局部重训练注意力模型
- **GLoRA**: 广义LoRA
- **AdaptLoRA**
- **PETC (2022)**
- **IA3**

### Prefix-tuning & Adapters的进一步探讨
- **Refine in LLMs**: 提炼在大型语言模型中
- **Project down & project up**: 降维与升维操作
- **Insertion Form - added type**: 插入形式
- **Manifested representation**: 表现形式
- **Composition function**: 组合函数
- **MAM Adapters**:
  - 对模型进行微调以适应更加特殊的需求

这些笔记显然是对先进的深度学习技术的探讨和记录，可能对进行复杂模型适配与优化的研究者或开发者特别有用。